# Quantum Teleportaton and Entanglement Swapping Using Sympy
This first part of this notebook demonstrates teleportation of a quantum state following the classic paper in Physical Review Letters: *Teleporting an Unknown State via Dual Classical and Einstein-Podolsky-Rosent Channels*, 29 March 1993, Vol. 70, #13, p. 1895

There are no numbers in this computation - it is all algebra perfomed automatically using sympy, numpy and sicpy.

Alice has a particle in a state unknown to her and she wishes to communicate that state to Bob by entangling it with a second particle and sending that particle to Bob. In order for Bob to decode the state, he needs two bits of classical information from Alice - her measurement results.

The steps are as follows:

<ol>
    <li> Alice prepares particles two and there in an entngled Bell state singlet state.</li>
    <li> Alice measures partciles one and two in the Bell basis, getting one of four possible outcomes.</li>
    <li> Bob receives the measurement results and depending on their value, chooses a gate to apply to particle three.</li>
    <li> Particle three is now in a replica state matching the original state of particle one.
</ol>

This Jupyter notebook computes the state of particle three before and after Bob applies his gate and demonstrates that the state of particle three is an exact replica of the initial state of particle one, verifying equations (5) and (6) in the paper. To do so, it defines Python classes for quantum states and operators and overloads the multiplication operator to perform quantum operations and tensor products of quantum operators.

The second part of this notebook, Entanglement Swapping, applies the teleportation technique to a more complicated
situation: the quantum Internet. There is a transmitting station, a receiving station and a quantum repeater between them. There are five qubits - the data qubit, three link qubits and the received qubit, which is placed in 
an exact replica of the data qubit. In accordance with the no-cloning principle, the data qubit is destroyed when 
it is transmitted.

In [6]:
import numpy as np
from scipy import linalg
import sympy as sp
from sympy import Matrix

# Extract the state of a particle from its density matrix by getting its eigenvectors. Only the states that have
# nonzero probability are retained.
def get_states(rho):
    m = Matrix(rho)
    states = []
    for i, triple in enumerate(m.eigenvects()):
        p = triple[0]
        multiplicity = triple[1]
        if p != 0:
            evects = triple[2]
            for evec in evects:
                evec1 = []
                for component in evec:
                    evec1.append(component)
                states.append(np.array(evec1))
    return np.array(states)

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

## Initial State
$\ket{\Psi}=a\ket{0}+b\ket{1}$

In [7]:
def initial_state():
    a, b = sp.symbols('a b')
    state = [a,b,0,0,0,0,0,0]
    return qstate(np.array(state))

## Partial Trace
After Alice has performed her measurement, the particles are in an entangled state. Bob needs to examine the state
of particle three in order to apply a gate to it. We simulate this by computing the partial trace of the three-particle density matrix over particles one and two.

In [8]:
def partial_trace_first_subsystem(rho,n,m):
    """
    Partial trace over second subsystem
    n=#qubits in first subsyste
    m=# qubits in second subsystem
    n+m=dim
    """
    n=2**n
    m=2**m
    return np.trace(rho.reshape(m,n,m,n), axis1=1, axis2=3)
def partial_trace_second_subsystem(rho,n,m):
    """
    Partial trace over first subsystem
    n=#qubits in first subsyste
    m=# qubits in second subsystem
    n+m=dim

    """
    n=2**n
    m=2**m
    return np.trace(rho.reshape(m,n,m,n), axis1=0, axis2=2)

## Python Classes for States and Operators
"tensor" represents a quantum operator and multiplication, \*, implements the tensor product of two operators acting on subspaces of the system Hilber space, using the Kronecker (matrix) product. "qstate" represents a quantum state and multiplication of a tensor by a state implements or simulates the action of an operator on a quantum state. The quantum method braket forms the outer product of a ket with a bra, for constructing projection operators and density matrices.

$A\otimes B \rightarrow$ a*b

$A \ket{\Psi} \rightarrow$ A*psi

$\ket{\Psi}\bra{\Phi} \rightarrow$ psi.braket(phi)

In [9]:
class tensor(object):
    def __init__(self, val):
        self.val = val
    def __mul__(self,other):
        if type(other)==tensor:
            return tensor(linalg.kron(self.val, other.val))
        elif type(other)==qstate:
            return qstate(np.matmul(other.val,np.conj(self.val)))
    def __add__(self,other):
        return tensor(self.val+other.val)
class qstate(object):
    def __init__(self,val):
        self.val = val
    def __rmul__(self, other):
        return qstate(np.matmul(self,other.val))
    def braket(self,other):
        return tensor(np.outer(other.val,np.conj(self.val)))

In [10]:
# Quantum gates
identity = tensor(np.array([[1,0],[0,1]]))
x_gate = tensor(np.array([[0,1],[1,0]]))
y_gate = tensor(np.array([[0,-complex(0,1)],[complex(0,1),0]], dtype=complex))
z_gate = tensor(np.array([[1,0],[0,-1]]))

# Bell states in the Bell basis (diagnonal)
phi_plus = qstate(np.array([1,0,0,0]))
phi_minus = qstate(np.array([0,1,0,0]))
psi_plus = qstate(np.array([0,0,1,0]))
psi_minus = qstate(np.array([0,0,0,1]))

# Bell states in the computational basis
phi_plus_c = qstate(np.array([1,0,0,1]/np.sqrt(2)))
phi_minus_c = qstate(np.array([1,0,0,-1]/np.sqrt(2)))
psi_plus_c = qstate(np.array([0,1,1,0]/np.sqrt(2)))
psi_minus_c = qstate(np.array([0,-1,1,0]/np.sqrt(2)))

# The quantum operation used by Alice to perform her measurements in the Bell basis
gate00 = phi_plus.braket(phi_plus_c)
gate10 = phi_minus.braket(phi_minus_c)
gate01 = psi_plus.braket(psi_plus_c)
gate11 = psi_minus.braket(psi_minus_c)
bell_gate = gate00+gate01+gate10+gate11

# The projection operator Alice uses to place particles two and three in the Psi minus Bell state

entangle_gate = psi_minus_c.braket(phi_plus)*identity
state = entangle_gate * initial_state()

# Apply the Bell operator to the particles one and two, leaving particle three unaffected
# In forming the tensor product, list the operators in reverse order (last subspace/last qubits first)
state = identity * bell_gate * state

# Alice's measurement with four possible outcomes
for i,psi in enumerate([phi_plus, phi_minus, psi_plus, psi_minus]): #The four outcomes
    proj = psi.braket(psi)
    proj = identity * proj
    state1 = proj * state
    
    # Bob examines the state of third particle. Simulate by forming density matrix and taking partial trace
    rho1= state1.braket(state1)
    rho1 = partial_trace_first_subsystem(rho1.val,2,1)
    state1 = get_states(rho1)[0]
    a,b = sp.symbols('a b')
    
    # Depending on the measurement outcome (i), Bob selects and applies a quantum operation
    if i == 0: #phi_plus
        gate = y_gate
    elif i == 1: #phi_minus
        gate = x_gate
    elif i == 2: #psi_plus
        gate = z_gate
    else: # psi_minus
        gate = identity
    state2 = qstate(get_states(rho1)[0])
    state2 = gate * state2
    
    # Verify that the final state is parallel to the initial state (cross prduct is zero)
    assert (state2.val[0]*b-state2.val[1]*a==0)


## Entanglement Swapping

In [11]:
def initial_state():
    a, b = sp.symbols('a b')
    state = np.zeros(32, dtype=type(a))
    state[0]=a
    state[1]=b
    return qstate(state)

In [12]:
# Quantum repeater measures in the Bell basis. Simlate by constructing the transformation from the Bell
# (diagonal) basis to the computational basis
gate00 = phi_plus_c.braket(phi_plus)
gate10 = phi_minus_c.braket(phi_minus)
gate01 = psi_plus_c.braket(psi_plus)
gate11 = psi_minus_c.braket(psi_minus)
bell_gate = gate00+gate10+gate01+gate11

# entangle qubits 1 and 2, leave 0, 3 and 4 unchanged
state = identity*identity*bell_gate*identity*initial_state()

# entangle qubits 3 and 4, leave 0,1 and 2 unchanged
state = bell_gate*identity*identity*identity*state

# All possible outcomes for the measurement of qubits 1 and 2
for i1,psi in enumerate([phi_plus_c, phi_minus_c, psi_plus_c, psi_minus_c]):
    
    proj = identity*psi.braket(psi)*identity*identity
    state1 = proj*state

    # All possible outcomes for the measurement of qubits 0 and 1
    for i2, psi2 in enumerate([phi_plus_c, phi_minus_c, psi_plus_c, psi_minus_c]):

        proj = identity*identity*identity*psi2.braket(psi2)
        state2 = proj * state1
        
        # Form density matrix and extract state
        rho1= state2.braket(state2).val
        rho1 = partial_trace_first_subsystem(rho1,4,1)
        state2 = qstate(get_states(rho1)[0])
        
        # Depending on the measurement outcomes, the receiver applies a gate to the link qubit.
        if i1==0:
            if i2 == 0:
                gate = None
            elif i2 == 1:
                gate = z_gate
            elif i2 == 2:
                gate = x_gate
            else:
                gate = y_gate
        elif i1==1:
            if i2 == 0:
                gate = z_gate
            elif i2 == 1:
                gate = None
            elif i2 == 2:
                gate = y_gate
            else:
                gate = x_gate

        elif i1 == 2:
            if i2 == 0:
                gate = x_gate
            elif i2 == 1:
                gate = y_gate
            elif i2 == 2:
                gate = None
            else:
                gate = z_gate
        elif i1 == 3:
            if i2 == 0:
                gate = y_gate
            elif i2 == 1:
                gate = x_gate
            elif i2 == 2:
                gate = z_gate
            else:
                gate = None
        if gate is not None:
            state2 = gate * state2
            
        # Verify that the intial qubit and the reconstructed, repeated, qubit are parallel - zero cross product
        a,b = sp.symbols('a b')
        assert (state2.val[0]*b-state2.val[1]*a==0)
